In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix
)

RANDOM_STATE = 42

In [22]:
df = pd.read_csv("mail_l7_dataset.csv")

# Replace missing values
df = df.where(pd.notnull(df), "")

# Encodee labels
df.loc[df["Category"].str.lower().str.strip() == "spam", "Category"] = 0
df.loc[df["Category"].str.lower().str.strip() == "ham",  "Category"] = 1

df["Category"] = df["Category"].astype(int)
df.head(5)


,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [31]:
X = df["Message"].astype(str)
y = df["Category"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

Train size: 4457
Test size: 1115


In [32]:
tfidf = TfidfVectorizer(stop_words="english", lowercase=True)

X_train_features = tfidf.fit_transform(X_train)
X_test_features  = tfidf.transform(X_test)

print("Train shape:", X_train_features.shape)
print("Test shape:", X_test_features.shape)

Train shape: (4457, 7440)
Test shape: (1115, 7440)


In [33]:
lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
lr.fit(X_train_features, y_train)

lr_pred = lr.predict(X_test_features)

In [34]:
rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
rf.fit(X_train_features, y_train)

rf_pred = rf.predict(X_test_features)

In [35]:
nb = MultinomialNB()
nb.fit(X_train_features, y_train)

nb_pred = nb.predict(X_test_features)

In [36]:
def evaluate_model(name, y_true, y_pred):
    print(f"\n{name} Performance:")
    print("Accuracy :", round(accuracy_score(y_true, y_pred), 3))
    print("Precision:", round(precision_score(y_true, y_pred, pos_label=0), 3))
    print("Recall   :", round(recall_score(y_true, y_pred, pos_label=0), 3))
    print("F1-Score :", round(f1_score(y_true, y_pred, pos_label=0), 3))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

In [37]:
evaluate_model("Logistic Regression", y_test, lr_pred)
evaluate_model("Random Forest", y_test, rf_pred)
evaluate_model("Naive Bayes", y_test, nb_pred)


Logistic Regression Performance:
Accuracy : 0.968
Precision: 1.0
Recall   : 0.758
F1-Score : 0.863
Confusion Matrix:
 [[113  36]
 [  0 966]]

Random Forest Performance:
Accuracy : 0.983
Precision: 1.0
Recall   : 0.872
F1-Score : 0.932
Confusion Matrix:
 [[130  19]
 [  0 966]]

Naive Bayes Performance:
Accuracy : 0.977
Precision: 1.0
Recall   : 0.826
F1-Score : 0.904
Confusion Matrix:
 [[123  26]
 [  0 966]]


In [38]:
messages = [
    "Free entry in 2 a weekly competition!",
    "I will meet you at the cafe tomorrow",
    "Congratulations, you won a free ticket"
]

def predict_message(text):
    vector = tfidf.transform([text])
    print("\nMessage:", text)
    print("LR :", "Spam" if lr.predict(vector)[0] == 0 else "Ham")
    print("RF :", "Spam" if rf.predict(vector)[0] == 0 else "Ham")
    print("NB :", "Spam" if nb.predict(vector)[0] == 0 else "Ham")

for msg in messages:
    predict_message(msg)


Message: Free entry in 2 a weekly competition!
LR : Ham
RF : Ham
NB : Spam

Message: I will meet you at the cafe tomorrow
LR : Ham
RF : Ham
NB : Ham

Message: Congratulations, you won a free ticket
LR : Ham
RF : Ham
NB : Ham
